# Pandas 4

---

## Content

- Multi-indexing
- Melting
  - `pd.melt()`
- Pivoting
  - `pd.pivot()`
  - `pd.pivot_table()`
- Binning
  - `pd.cut()`

---

### Multi-Indexing

In [ ]:
!pip install --upgrade gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3


In [1]:
!gdown 1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd
!gdown 1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm

Downloading...
From: https://drive.google.com/uc?id=1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd
To: /content/movies.csv
100% 112k/112k [00:00<00:00, 18.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm
To: /content/directors.csv
100% 65.4k/65.4k [00:00<00:00, 60.7MB/s]


In [30]:
import pandas as pd
import numpy as np

movies = pd.read_csv('movies.csv', index_col=0)
directors = pd.read_csv('directors.csv', index_col=0)

data = movies.merge(directors, how='left', left_on='director_id',right_on='id')
data.drop(['director_id','id_y'],axis=1,inplace=True)

**Which director according to you should be considered as most productive?**

- Should we decide based on the **number of movies** directed?
- Or take the **quality of the movies** into consideration as well?
- Or maybe look at the the **amount of business** the movie is doing?

To simplify, let's calculate who has directed maximum number of movies.

In [31]:
# productive based on no.of movies
data["director_name"].value_counts()

,count
director_name,
Steven Spielberg,26
Martin Scorsese,19
Clint Eastwood,19
Woody Allen,18
Ridley Scott,16
...,...
Tim Hill,5
Jonathan Liebesman,5
Roman Polanski,5


In [32]:
data.groupby(['director_name'])['title'].count().sort_values(ascending=False)

,title
director_name,
Steven Spielberg,26
Clint Eastwood,19
Martin Scorsese,19
Woody Allen,18
Robert Rodriguez,16
...,...
Paul Weitz,5
John Madden,5
Paul Verhoeven,5


`Steven Spielberg` has directed maximum number of movies.

**But does it make `Steven` the most productive director?**

- Chances are, he might be active for more years than the other directors.

**Calculating the active years for every director?**

- We can subtract both `min` and `max` of year.

In [ ]:
# Productive as : number of movies/year

In [33]:
data_agg = data.groupby(['director_name'])[["year", "title"]].aggregate({"year":['min','max'], "title": "count"})
data_agg

year       title
                              min   max count
director_name                                
Adam McKay                   2004  2015     6
Adam Shankman                2001  2012     8
Alejandro González Iñárritu  2000  2015     6
Alex Proyas                  1994  2016     5
Alexander Payne              1999  2013     5
...                           ...   ...   ...
Wes Craven                   1984  2011    10
Wolfgang Petersen            1981  2006     7
Woody Allen                  1977  2013    18
Zack Snyder                  2004  2016     7
Zhang Yimou                  2002  2014     6

[199 rows x 3 columns]

Notice,
- `director_name` column has turned into **row labels**.
- There are multiple levels for the column names.

This is called a **Multi-index DataFrame**.

- It can have **multiple indexes along a dimension**.
  - The no. of dimensions remain same though.
- Multi-level indexes are **possible both for rows and columns**.

In [34]:
data_agg.columns

MultiIndex([( 'year',   'min'),
            ( 'year',   'max'),
            ('title', 'count')],
           )

The level-1 column names are `year` and `title`.

**What would happen if we print the column `year` of this multi-index dataframe?**

In [35]:
data_agg["year"]

,min,max
director_name,,
Adam McKay,2004,2015
Adam Shankman,2001,2012
Alejandro González Iñárritu,2000,2015
Alex Proyas,1994,2016
Alexander Payne,1999,2013
...,...,...
Wes Craven,1984,2011
Wolfgang Petersen,1981,2006
Woody Allen,1977,2013


In [36]:
data_agg["year"]["min"]

,min
director_name,
Adam McKay,2004
Adam Shankman,2001
Alejandro González Iñárritu,2000
Alex Proyas,1994
Alexander Payne,1999
...,...
Wes Craven,1984
Wolfgang Petersen,1981
Woody Allen,1977


**How can we convert multi-level back to only one level of columns?**

- e.g. `year_min`, `year_max`, `title_count`

In [ ]:
data_agg = data.groupby(['director_name'])[["year","title"]].aggregate(
    {"year":['min', 'max'], "title": "count"})

In [ ]:
data_agg.columns = ['_'.join(col) for col in data_agg.columns]
data_agg

,year_min,year_max,title_count
director_name,,,
Adam McKay,2004,2015,6
Adam Shankman,2001,2012,8
Alejandro González Iñárritu,2000,2015,6
Alex Proyas,1994,2016,5
Alexander Payne,1999,2013,5
...,...,...,...
Wes Craven,1984,2011,10
Wolfgang Petersen,1981,2006,7
Woody Allen,1977,2013,18


Since these were tuples, we can just join them.

In [ ]:
# Simplified version for above
data.groupby('director_name')[['year', 'title']].aggregate(
    year_max=('year','max'),
    year_min=('year','min'),
    title_count=('title','count')
)

,year_max,year_min,title_count
director_name,,,
Adam McKay,2015,2004,6
Adam Shankman,2012,2001,8
Alejandro González Iñárritu,2015,2000,6
Alex Proyas,2016,1994,5
Alexander Payne,2013,1999,5
...,...,...,...
Wes Craven,2011,1984,10
Wolfgang Petersen,2006,1981,7
Woody Allen,2013,1977,18


The columns look good, but we may want to turn back the row labels into a proper column as well.

**Converting row labels into a column using `reset_index` -**

In [ ]:
data_agg.reset_index()

,director_name,year_min,year_max,title_count
0,Adam McKay,2004,2015,6
1,Adam Shankman,2001,2012,8
2,Alejandro González Iñárritu,2000,2015,6
3,Alex Proyas,1994,2016,5
4,Alexander Payne,1999,2013,5
...,...,...,...,...
194,Wes Craven,1984,2011,10
195,Wolfgang Petersen,1981,2006,7
196,Woody Allen,1977,2013,18
197,Zack Snyder,2004,2016,7


**Using the new features, can we find the most productive director?**

1. First calculate how many years the director has been active.

In [ ]:
data_agg["yrs_active"] = data_agg["year_max"] - data_agg["year_min"]
data_agg

,year_min,year_max,title_count,yrs_active
director_name,,,,
Adam McKay,2004,2015,6,11
Adam Shankman,2001,2012,8,11
Alejandro González Iñárritu,2000,2015,6,15
Alex Proyas,1994,2016,5,22
Alexander Payne,1999,2013,5,14
...,...,...,...,...
Wes Craven,1984,2011,10,27
Wolfgang Petersen,1981,2006,7,25
Woody Allen,1977,2013,18,36


2. Then calculate rate of directing movies by `title_count`/`yrs_active`.

In [ ]:
data_agg["movie_per_yr"] = data_agg["title_count"] / data_agg["yrs_active"]
data_agg

,year_min,year_max,title_count,yrs_active,movie_per_yr
director_name,,,,,
Adam McKay,2004,2015,6,11,0.545455
Adam Shankman,2001,2012,8,11,0.727273
Alejandro González Iñárritu,2000,2015,6,15,0.400000
Alex Proyas,1994,2016,5,22,0.227273
Alexander Payne,1999,2013,5,14,0.357143
...,...,...,...,...,...
Wes Craven,1984,2011,10,27,0.370370
Wolfgang Petersen,1981,2006,7,25,0.280000
Woody Allen,1977,2013,18,36,0.500000


3. Finally, sort the values.

In [ ]:
data_agg.sort_values("movie_per_yr", ascending=False)

,year_min,year_max,title_count,yrs_active,movie_per_yr
director_name,,,,,
Tyler Perry,2006,2013,9,7,1.285714
Jason Friedberg,2006,2010,5,4,1.250000
Shawn Levy,2002,2014,11,12,0.916667
Robert Rodriguez,1992,2014,16,22,0.727273
Adam Shankman,2001,2012,8,11,0.727273
...,...,...,...,...,...
Lawrence Kasdan,1985,2012,5,27,0.185185
Luc Besson,1985,2014,5,29,0.172414
Robert Redford,1980,2010,5,30,0.166667


**Conclusion:**

- `Tyler Perry` turns out to be truly the most productive director.

---

### PFizer data

For this topic we will be using data of few drugs being developed by **PFizer**.

Dataset: https://drive.google.com/file/d/173A59xh2mnpmljCCB9bhC4C5eP2IS6qZ/view?usp=sharing

In [7]:
!gdown 173A59xh2mnpmljCCB9bhC4C5eP2IS6qZ

Downloading...
From: https://drive.google.com/uc?id=173A59xh2mnpmljCCB9bhC4C5eP2IS6qZ
To: /content/Pfizer_1.csv
100% 1.51k/1.51k [00:00<00:00, 5.20MB/s]


**What is the data about?**
- Temperature (K)
- Pressure (P)

The data is recorded after an **interval of 1 hour** everyday to monitor the drug stability in a drug development test.

These data points are therefore used to **identify the optimal set of values of parameters** for the stability of the drugs.

Let's explore this dataset -


In [9]:
data = pd.read_csv('Pfizer_1.csv')
data

,Date,Drug_Name,Parameter,1:30:00,2:30:00,3:30:00,4:30:00,5:30:00,6:30:00,7:30:00,8:30:00,9:30:00,10:30:00,11:30:00,12:30:00
0,15-10-2020,diltiazem hydrochloride,Temperature,23.0,22.0,NaN,21.0,21.0,22,23.0,21.0,22.0,20,20.0,21
1,15-10-2020,diltiazem hydrochloride,Pressure,12.0,13.0,NaN,11.0,13.0,14,16.0,16.0,24.0,18,19.0,20
2,15-10-2020,docetaxel injection,Temperature,NaN,17.0,18.0,NaN,17.0,18,NaN,NaN,23.0,23,25.0,25
3,15-10-2020,docetaxel injection,Pressure,NaN,22.0,22.0,NaN,22.0,23,NaN,NaN,27.0,26,29.0,28
4,15-10-2020,ketamine hydrochloride,Temperature,24.0,NaN,NaN,27.0,NaN,26,25.0,24.0,23.0,22,21.0,20
5,15-10-2020,ketamine hydrochloride,Pressure,8.0,NaN,NaN,7.0,NaN,9,10.0,11.0,10.0,9,9.0,11
6,16-10-2020,diltiazem hydrochloride,Temperature,34.0,35.0,36.0,36.0,37.0,38,37.0,38.0,39.0,40,NaN,42
7,16-10-2020,diltiazem hydrochloride,Pressure,18.0,19.0,20.0,21.0,22.0,23,24.0,25.0,25.0,24,NaN,27
8,16-10-2020,docetaxel injection,Temperature,46.0,47.0,NaN,48.0,48.0,49,50.0,52.0,55.0,56,57.0,58
9,16-10-2020,docetaxel injection,Pressure,23.0,24.0,NaN,25.0,26.0,27,28.0,29.0,28.0,28,29.0,30


In [ ]:
data.info()
# this is wide format data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       18 non-null     object 
 1   Drug_Name  18 non-null     object 
 2   Parameter  18 non-null     object 
 3   1:30:00    16 non-null     float64
 4   2:30:00    16 non-null     float64
 5   3:30:00    12 non-null     float64
 6   4:30:00    14 non-null     float64
 7   5:30:00    16 non-null     float64
 8   6:30:00    18 non-null     int64  
 9   7:30:00    16 non-null     float64
 10  8:30:00    14 non-null     float64
 11  9:30:00    16 non-null     float64
 12  10:30:00   18 non-null     int64  
 13  11:30:00   16 non-null     float64
 14  12:30:00   18 non-null     int64  
dtypes: float64(9), int64(3), object(3)
memory usage: 2.2+ KB


In [10]:
data.shape

(18, 15)

---

## Melting

As we saw earlier, the dataset has **18 rows** and **15 columns**.

If you notice further, you'll see:
- The columns are `1:30:00`, `2:30:00`, `3:30:00`, ... so on.
- `Temperature` and `Pressure` of each date is in a separate row.

**Can we restructure our data into a better format?**

- Maybe we can have a column for `time`, with `timestamps` as the column value.

**Where will the Temperature/Pressure values go?**

- We can similarly create one column containing the values of these parameters.
- "Melt" the timestamp column into two columns** - timestamp and corresponding values

**How can we restructure our data into having every row corresponding to a single reading?**

Melting - wide to long format

In [12]:
pd.melt(data, id_vars = ["Date", "Parameter", "Drug_Name"]) # we are saying melt the data except the columns provided in the brackets

,Date,Parameter,Drug_Name,variable,value
0,15-10-2020,Temperature,diltiazem hydrochloride,1:30:00,23.0
1,15-10-2020,Pressure,diltiazem hydrochloride,1:30:00,12.0
2,15-10-2020,Temperature,docetaxel injection,1:30:00,NaN
3,15-10-2020,Pressure,docetaxel injection,1:30:00,NaN
4,15-10-2020,Temperature,ketamine hydrochloride,1:30:00,24.0
...,...,...,...,...,...
211,17-10-2020,Pressure,diltiazem hydrochloride,12:30:00,14.0
212,17-10-2020,Temperature,docetaxel injection,12:30:00,23.0
213,17-10-2020,Pressure,docetaxel injection,12:30:00,28.0
214,17-10-2020,Temperature,ketamine hydrochloride,12:30:00,24.0


In [ ]:
# On 15th october for diltiazem hydrochloride what was the pressure recorded at 1:30

This converts our data from `wide` to `long` format.

Notice that the `id_vars` are set of variables which remain unmelted.

**How does `pd.melt()` work?**

- Pass in the **DataFrame**.
- Pass in the **column names that we don't want to melt**.

But we can provide better names to these new columns.

**How can we rename the columns "variable" and "value" as per our original dataframe?**

In [13]:
data_melt = pd.melt(data,id_vars = ['Date', 'Drug_Name', 'Parameter'],
            var_name = "time",
            value_name = 'reading')
data_melt

,Date,Drug_Name,Parameter,time,reading
0,15-10-2020,diltiazem hydrochloride,Temperature,1:30:00,23.0
1,15-10-2020,diltiazem hydrochloride,Pressure,1:30:00,12.0
2,15-10-2020,docetaxel injection,Temperature,1:30:00,NaN
3,15-10-2020,docetaxel injection,Pressure,1:30:00,NaN
4,15-10-2020,ketamine hydrochloride,Temperature,1:30:00,24.0
...,...,...,...,...,...
211,17-10-2020,diltiazem hydrochloride,Pressure,12:30:00,14.0
212,17-10-2020,docetaxel injection,Temperature,12:30:00,23.0
213,17-10-2020,docetaxel injection,Pressure,12:30:00,28.0
214,17-10-2020,ketamine hydrochloride,Temperature,12:30:00,24.0


**Conclusion:**

- The labels of the timestamp columns are conviniently **melted into a single column** - `time`
- It retained all the values in `reading` column.
- The labels of columns such as `1:30:00`, `2:30:00` have now become categories of the `variable` column.
- The values from columns we are melting are stored in the `value` column.

---

### Pivoting

Now suppose we want to convert our data back to the **wide format**.

The reason could be to maintain the structure for storing or some other purpose.

Notice,

- The variables `Date`, `Drug_Name` and `Parameter` will remain same.
- The column names will be extracted from the column `time`.
- The values will be extracted from the column `readings`.

**How can we restructure our data back to the original wide format?**

pivot: long to wide

In [ ]:
data_melt.pivot(index=['Date','Drug_Name','Parameter'],  # Columns used to make new frame’s index
                columns = 'time',                        # Column used to make new frame’s columns
                values='reading')                        # Column used for populating new frame’s values.

time                                            10:30:00  11:30:00  12:30:00  \
Date       Drug_Name               Parameter                                   
15-10-2020 diltiazem hydrochloride Pressure         18.0      19.0      20.0   
                                   Temperature      20.0      20.0      21.0   
           docetaxel injection     Pressure         26.0      29.0      28.0   
                                   Temperature      23.0      25.0      25.0   
           ketamine hydrochloride  Pressure          9.0       9.0      11.0   
                                   Temperature      22.0      21.0      20.0   
16-10-2020 diltiazem hydrochloride Pressure         24.0       NaN      27.0   
                                   Temperature      40.0       NaN      42.0   
           docetaxel injection     Pressure         28.0      29.0      30.0   
                                   Temperature      56.0      57.0      58.0   
           ketamine hydrochloride  Pressure         16.0      17.0      18.0   
                                   Temperature      13.0      14.0      15.0   
17-10-2020 diltiazem hydrochloride Pressure         11.0      13.0      14.0   
                                   Temperature      14.0      11.0      10.0   
           docetaxel injection     Pressure         28.0      29.0      28.0   
                                   Temperature      21.0      22.0      23.0   
           ketamine hydrochloride  Pressure         13.0      14.0      15.0   
                                   Temperature      22.0      23.0      24.0   

time                                            1:30:00  2:30:00  3:30:00  \
Date       Drug_Name               Parameter                                
15-10-2020 diltiazem hydrochloride Pressure        12.0     13.0      NaN   
                                   Temperature     23.0     22.0      NaN   
           docetaxel injection     Pressure         NaN     22.0     22.0   
                                   Temperature      NaN     17.0     18.0   
           ketamine hydrochloride  Pressure         8.0      NaN      NaN   
                                   Temperature     24.0      NaN      NaN   
16-10-2020 diltiazem hydrochloride Pressure        18.0     19.0     20.0   
                                   Temperature     34.0     35.0     36.0   
           docetaxel injection     Pressure        23.0     24.0      NaN   
                                   Temperature     46.0     47.0      NaN   
           ketamine hydrochloride  Pressure        12.0     12.0     13.0   
                                   Temperature      8.0      9.0     10.0   
17-10-2020 diltiazem hydrochloride Pressure         3.0      4.0      4.0   
                                   Temperature     20.0     19.0     19.0   
           docetaxel injection     Pressure        20.0     22.0     22.0   
                                   Temperature     12.0     13.0     14.0   
           ketamine hydrochloride  Pressure         8.0      9.0     10.0   
                                   Temperature     13.0     14.0     15.0   

time                                            4:30:00  5:30:00  6:30:00  \
Date       Drug_Name               Parameter                                
15-10-2020 diltiazem hydrochloride Pressure        11.0     13.0     14.0   
                                   Temperature     21.0     21.0     22.0   
           docetaxel injection     Pressure         NaN     22.0     23.0   
                                   Temperature      NaN     17.0     18.0   
           ketamine hydrochloride  Pressure         7.0      NaN      9.0   
                                   Temperature     27.0      NaN     26.0   
16-10-2020 diltiazem hydrochloride Pressure        21.0     22.0     23.0   
                                   Temperature     36.0     37.0     38.0   
           docetaxel injection     Pressure        25.0     26.0     27.0   
           

Notice that `pivot()` is the exact opposite of `melt()`.

We are getting **multiple indices** here, but we can get single index again using `reset_index()`.

In [ ]:
data_melt.pivot(index=['Date','Drug_Name','Parameter'],
                columns = 'time',
                values='reading').reset_index()

time,Date,Drug_Name,Parameter,10:30:00,11:30:00,12:30:00,1:30:00,2:30:00,3:30:00,4:30:00,5:30:00,6:30:00,7:30:00,8:30:00,9:30:00
0,15-10-2020,diltiazem hydrochloride,Pressure,18.0,19.0,20.0,12.0,13.0,NaN,11.0,13.0,14.0,16.0,16.0,24.0
1,15-10-2020,diltiazem hydrochloride,Temperature,20.0,20.0,21.0,23.0,22.0,NaN,21.0,21.0,22.0,23.0,21.0,22.0
2,15-10-2020,docetaxel injection,Pressure,26.0,29.0,28.0,NaN,22.0,22.0,NaN,22.0,23.0,NaN,NaN,27.0
3,15-10-2020,docetaxel injection,Temperature,23.0,25.0,25.0,NaN,17.0,18.0,NaN,17.0,18.0,NaN,NaN,23.0
4,15-10-2020,ketamine hydrochloride,Pressure,9.0,9.0,11.0,8.0,NaN,NaN,7.0,NaN,9.0,10.0,11.0,10.0
5,15-10-2020,ketamine hydrochloride,Temperature,22.0,21.0,20.0,24.0,NaN,NaN,27.0,NaN,26.0,25.0,24.0,23.0
6,16-10-2020,diltiazem hydrochloride,Pressure,24.0,NaN,27.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,25.0
7,16-10-2020,diltiazem hydrochloride,Temperature,40.0,NaN,42.0,34.0,35.0,36.0,36.0,37.0,38.0,37.0,38.0,39.0
8,16-10-2020,docetaxel injection,Pressure,28.0,29.0,30.0,23.0,24.0,NaN,25.0,26.0,27.0,28.0,29.0,28.0
9,16-10-2020,docetaxel injection,Temperature,56.0,57.0,58.0,46.0,47.0,NaN,48.0,48.0,49.0,50.0,52.0,55.0


In [14]:
data_melt.head()

,Date,Drug_Name,Parameter,time,reading
0,15-10-2020,diltiazem hydrochloride,Temperature,1:30:00,23.0
1,15-10-2020,diltiazem hydrochloride,Pressure,1:30:00,12.0
2,15-10-2020,docetaxel injection,Temperature,1:30:00,NaN
3,15-10-2020,docetaxel injection,Pressure,1:30:00,NaN
4,15-10-2020,ketamine hydrochloride,Temperature,1:30:00,24.0


Now if you notice,
- We are using 2 rows to log readings for a single experiment. one for temperature, one for pressure

**Can we further restructure our data into dividing the `Parameter` column into T/P?**

- A format like `Date | time | Drug_Name | Pressure | Temperature` would be suitable.
- We want to **split one single column into multiple columns**.

**How can we divide the `Parameter` column again?**

In [17]:
data_tidy = data_melt.pivot(index=['Date','time', 'Drug_Name'],
                            columns = 'Parameter',
                            values='reading')
data_tidy

Parameter                                    Pressure  Temperature
Date       time     Drug_Name                                     
15-10-2020 10:30:00 diltiazem hydrochloride      18.0         20.0
                    docetaxel injection          26.0         23.0
                    ketamine hydrochloride        9.0         22.0
           11:30:00 diltiazem hydrochloride      19.0         20.0
                    docetaxel injection          29.0         25.0
...                                               ...          ...
17-10-2020 8:30:00  docetaxel injection          26.0         19.0
                    ketamine hydrochloride       11.0         20.0
           9:30:00  diltiazem hydrochloride       9.0         13.0
                    docetaxel injection          27.0         20.0
                    ketamine hydrochloride       12.0         21.0

[108 rows x 2 columns]

Notice that a **multi-index** dataframe has been created.

We can use `reset_index()` to remove the multi-index.

In [19]:
data_tidy = data_tidy.reset_index()
data_tidy

,Date,time,Drug_Name,Pressure,Temperature
0,15-10-2020,10:30:00,diltiazem hydrochloride,18.0,20.0
1,15-10-2020,10:30:00,docetaxel injection,26.0,23.0
2,15-10-2020,10:30:00,ketamine hydrochloride,9.0,22.0
3,15-10-2020,11:30:00,diltiazem hydrochloride,19.0,20.0
4,15-10-2020,11:30:00,docetaxel injection,29.0,25.0
...,...,...,...,...,...
103,17-10-2020,8:30:00,docetaxel injection,26.0,19.0
104,17-10-2020,8:30:00,ketamine hydrochloride,11.0,20.0
105,17-10-2020,9:30:00,diltiazem hydrochloride,9.0,13.0
106,17-10-2020,9:30:00,docetaxel injection,27.0,20.0


We can rename our ```index``` column from `Parameter` to simply `None`.

In [20]:
data_tidy.columns.name = None
data_tidy.head()

,Date,time,Drug_Name,Pressure,Temperature
0,15-10-2020,10:30:00,diltiazem hydrochloride,18.0,20.0
1,15-10-2020,10:30:00,docetaxel injection,26.0,23.0
2,15-10-2020,10:30:00,ketamine hydrochloride,9.0,22.0
3,15-10-2020,11:30:00,diltiazem hydrochloride,19.0,20.0
4,15-10-2020,11:30:00,docetaxel injection,29.0,25.0


---

### Pivot Table

Now suppose we want to find some insights, like **mean temperature day-wise**.

**Can we use pivot to find the day-wise mean value of temperature for each drug?**

In [21]:
data_tidy.pivot(index=['Drug_Name'],
                columns = 'Date',
                values=['Temperature'])

ValueError: Index contains duplicate entries, cannot reshape

**Why did we get an error?**

- We need to find the **average** of temperature values throughout a day.
- If you notice, the error shows **duplicate entries**.

Hence, the index values should be unique entry for each row.

**What can we do to get our required mean values then?**


In [22]:
pd.pivot_table(data_tidy, index='Drug_Name', columns='Date', values=['Temperature'], aggfunc=np.mean)

<ipython-input-22-c8107c109de8>:1: FutureWarning: The provided callable <function mean at 0x79a9c3334310> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pd.pivot_table(data_tidy, index='Drug_Name', columns='Date', values=['Temperature'], aggfunc=np.mean)


Temperature                      
Date                     15-10-2020 16-10-2020 17-10-2020
Drug_Name                                                
diltiazem hydrochloride   21.454545  37.454545  15.636364
docetaxel injection       20.750000  51.454545  17.500000
ketamine hydrochloride    23.555556  11.500000  18.500000

This function is similar to `pivot()`, with an extra feature of an aggregator.

**How does `pivot_table()` work?**

- The initial parameters are same as what we use in `pivot()`.
- As an extra parameter, we pass the **type of aggregator**.

**Note:**

- We could have done this using `groupby` too.
- In fact, `pivot_table` uses `groupby` in the backend to group the data and perform the aggregration.
- The only difference is in the type of output we get using both the functions.

**Similarly, what if we want to find the minimum values of temperature and pressure on a particular date?**

In [23]:
pd.pivot_table(data_tidy, index='Drug_Name', columns='Date', values=['Temperature', 'Pressure'], aggfunc=np.min)

<ipython-input-23-8d9c0df1b3c3>:1: FutureWarning: The provided callable <function min at 0x79a9c3313ac0> is currently using DataFrameGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  pd.pivot_table(data_tidy, index='Drug_Name', columns='Date', values=['Temperature', 'Pressure'], aggfunc=np.min)


Pressure                       Temperature  \
Date                    15-10-2020 16-10-2020 17-10-2020  15-10-2020   
Drug_Name                                                              
diltiazem hydrochloride       11.0       18.0        3.0        20.0   
docetaxel injection           22.0       23.0       20.0        17.0   
ketamine hydrochloride         7.0       12.0        8.0        20.0   

                                               
Date                    16-10-2020 17-10-2020  
Drug_Name                                      
diltiazem hydrochloride       34.0       10.0  
docetaxel injection           46.0       12.0  
ketamine hydrochloride         8.0       13.0

In [26]:
# Figure out different aggregations on different values ?
pd.pivot_table(data_tidy, index="Drug_Name", columns="Date", aggfunc={ "Temperature": np.mean, "Pressure": np.min})

<ipython-input-26-48f1e981d51e>:2: FutureWarning: The provided callable <function mean at 0x79a9c3334310> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pd.pivot_table(data_tidy, index="Drug_Name", columns="Date", aggfunc={ "Temperature": np.mean, "Pressure": np.min})
<ipython-input-26-48f1e981d51e>:2: FutureWarning: The provided callable <function min at 0x79a9c3313ac0> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  pd.pivot_table(data_tidy, index="Drug_Name", columns="Date", aggfunc={ "Temperature": np.mean, "Pressure": np.min})


Pressure                       Temperature  \
Date                    15-10-2020 16-10-2020 17-10-2020  15-10-2020   
Drug_Name                                                              
diltiazem hydrochloride       11.0       18.0        3.0   21.454545   
docetaxel injection           22.0       23.0       20.0   20.750000   
ketamine hydrochloride         7.0       12.0        8.0   23.555556   

                                               
Date                    16-10-2020 17-10-2020  
Drug_Name                                      
diltiazem hydrochloride  37.454545  15.636364  
docetaxel injection      51.454545  17.500000  
ketamine hydrochloride   11.500000  18.500000

---

### Binning

Sometimes, we would want our data to be in **categorical** form instead of **continuous/numerical**.

- Let's say, instead of knowing specific test values of a month, I want to know its type.
- Depending on the level of granularity, we want to have - Low, Medium, High, Very High.

**How can we derive bins/buckets from continous data?**

- use `pd.cut()`

Let's try to use this on our `Temperature` column to categorise the data into bins.

But to define categories, let's first check `min` and `max` temperature values.

In [ ]:
data_tidy

,Date,time,Drug_Name,Pressure,Temperature,Temperature_avg,Pressure_avg
0,15-10-2020,10:30:00,diltiazem hydrochloride,18.0,20.0,24.848485,15.424242
1,15-10-2020,10:30:00,docetaxel injection,26.0,23.0,30.387097,25.483871
2,15-10-2020,10:30:00,ketamine hydrochloride,9.0,22.0,17.709677,11.935484
3,15-10-2020,11:30:00,diltiazem hydrochloride,19.0,20.0,24.848485,15.424242
4,15-10-2020,11:30:00,docetaxel injection,29.0,25.0,30.387097,25.483871
...,...,...,...,...,...,...,...
103,17-10-2020,8:30:00,docetaxel injection,26.0,19.0,30.387097,25.483871
104,17-10-2020,8:30:00,ketamine hydrochloride,11.0,20.0,17.709677,11.935484
105,17-10-2020,9:30:00,diltiazem hydrochloride,9.0,13.0,24.848485,15.424242
106,17-10-2020,9:30:00,docetaxel injection,27.0,20.0,30.387097,25.483871


In [ ]:
print(data_tidy['Temperature'].min(), data_tidy['Temperature'].max())

8.0 58.0


Here,
- Min value = 8
- Max value = 58

Lets's keep some buffer for future values and take the range from 5-60 (instead of 8-58).

We'll divide this data into **4 bins** of 10-15 values each.

In [ ]:
temp_points = [5, 20, 35, 50, 60]

temp_labels = ['low','medium','high','very_high'] # labels define the severity of the resultant output of the test

In [ ]:
data_tidy['temp_cat'] = pd.cut(data_tidy['Temperature'], bins=temp_points, labels=temp_labels)
data_tidy.head()

,Date,time,Drug_Name,Pressure,Temperature,Temperature_avg,Pressure_avg,temp_cat
0,15-10-2020,10:30:00,diltiazem hydrochloride,18.0,20.0,24.848485,15.424242,low
1,15-10-2020,10:30:00,docetaxel injection,26.0,23.0,30.387097,25.483871,medium
2,15-10-2020,10:30:00,ketamine hydrochloride,9.0,22.0,17.709677,11.935484,medium
3,15-10-2020,11:30:00,diltiazem hydrochloride,19.0,20.0,24.848485,15.424242,low
4,15-10-2020,11:30:00,docetaxel injection,29.0,25.0,30.387097,25.483871,medium


In [ ]:
data_tidy['temp_cat'].value_counts()

low          50
medium       38
high         15
very_high     5
Name: temp_cat, dtype: int64

**Note:** By default, `pd.cut()` creates intervals of the form (x, y] — which includes the right endpoint but excludes the left one.

---